In [ ]:
!pip install flask-ngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/DLBot_Build

/content/drive/My Drive/DLBot_Build


In [ ]:
!pwd

/content/drive/My Drive/DLBot_Build


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
50 documents
10 classes ['assistance_required', 'emergency', 'goodbye', 'greeting', 'hospital_search', 'medical_history', 'medication', 'options', 'pharmacy_search', 'thanks']
85 unique lemmatized words ["'s", ',', 'a', 'all', 'an', 'any', 'anyone', 'are', 'assistance', 'awesome', 'be', 'bye', 'can', 'chatting', 'check', 'could', 'day', 'do', 'drug', 'emergency', 'ever', 'find', 'for', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hospital', 'how', 'i', 'is', 'it', 'later', 'list', 'locate', 'me', 'medical', 'medication', 'medicine', 'my', "n't", 'nearby', 'nearest', 'need', 'next', 'nice', 'no', 'of', 'offered', 'open', 'or', 'pharmacy', 'please', 'prescribed', 'provide', 'quickly', 'record', 'search', 'see', 'show', '

In [ ]:
#imports
from flask_ngrok import run_with_ngrok
#
from flask import Flask, render_template, request
app = Flask(__name__)
run_with_ngrok(app)
#create chatbot
import nltk
import numpy as np
import random
import string 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
from keras.models import load_model
model = load_model('chatbot_model.h5')
import pickle
words = pickle.load(open("words.pkl", 'rb'))
#result = words.score(X_test, Y_test)
classes = pickle.load(open("classes.pkl", 'rb'))


data_file = open('intents.json').read()
intents = json.loads(data_file)

#chatbot functions created
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        if r[1]>=0.60:
            return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        else:
            return_list.append({"intent": "noanswer", "probability": str(r[1])})
    print(return_list)
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            context=i.get("context","")#even if contexts doesn't exist, it returns value "" rather than keyerror
            break
    return [result,context]

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res
#define sql function
def _power(num,n):
    return num**n
def retrieve_sql_table(tag):
    tag=str(tag[0])#redundant
    import json
    sendmail=0
    call=0
    with open('sqlmap.json') as json_file: 
        data = json.load(json_file)
    print(data)
    if tag in data:
        s=data[tag]
        import sqlite3
        conn = sqlite3.connect('mydatabase2.db')
        conn.create_function("POWER", 2, _power)
        cursorObj = conn.cursor()
        colheader=cursorObj.execute(s)
        l=cursorObj.fetchall()
        col_name=[i[0] for i in colheader.description]
        a=[]
        a.append(col_name)
        for i in l:
            a.append(i)
        import pandas as pd
        df=pd.DataFrame(a)
        df.columns = df.iloc[0]
        df = df[1:]
        from IPython.display import HTML
        sqltable=df.to_html()
        return sqltable
    else:
        return ""
    
#define app routes
@app.route("/")
def index():
    return render_template("index.html")
@app.route("/get")
#function for the bot response
def get_bot_response():
    userText = request.args.get('msg')
    #return "hi"#works
    #return str(userText) #also works
    #trial part
    bot_response=""
    user_response = str(userText)
    user_response=user_response.lower()
    bot_response=chatbot_response(user_response)[0]
    print("tag:"+str(chatbot_response(user_response)[1]))
    sql_response=retrieve_sql_table(chatbot_response(user_response)[1])
    bot_response+=sql_response
    print(str(sql_response))
    return str(bot_response)


if __name__ == "__main__":
    app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dd6bb7653aba.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Nov/2020 05:57:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2020 05:57:42] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2020 05:57:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Nov/2020 05:57:45] "GET /get?msg=hey HTTP/1.1" 200 -


[{'intent': 'greeting', 'probability': '0.9995844'}]
[{'intent': 'greeting', 'probability': '0.9995844'}]
tag:['']
[{'intent': 'greeting', 'probability': '0.9995844'}]
{'medication_module': 'Select * from Medication where Duration_in_weeks>3', 'search_medical_history': 'Select * from Medication', 'search_pharmacy_by_name': 'Select Name, POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) AS Distance from Hospitals order by distance limit 5', 'search_hospital_by_name': 'Select Name from Hospitals order by POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) limit 5'}



...

[{'intent': 'pharmacy_search', 'probability': '0.99999166'}]
[{'intent': 'pharmacy_search', 'probability': '0.99999166'}]
tag:['search_pharmacy_by_name']
[{'intent': 'pharmacy_search', 'probability': '0.99999166'}]
{'medication_module': 'Select * from Medication where Duration_in_weeks>3', 'search_medical_history': 'Select * from Medication', 'search_pharmacy_by_name': 'Select Name, POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) AS Distance from Hospitals order by distance limit 5', 'search_hospital_by_name': 'Select Name from Hospitals order by POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) limit 5'}
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>NAME</th>
      <th>Distance</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>BELAU NATIONAL HOSPITAL</td>
      <td>32.5523</td>
    </tr>
    <tr>
      <th>2</th>
      <td>COMMONW

127.0.0.1 - - [06/Nov/2020 07:15:57] "GET /get?msg=help%20me HTTP/1.1" 200 -


[{'intent': 'emergency', 'probability': '0.96288735'}]
[{'intent': 'emergency', 'probability': '0.96288735'}]
tag:['emergency']
[{'intent': 'emergency', 'probability': '0.96288735'}]
{'medication_module': 'Select * from Medication where Duration_in_weeks>3', 'search_medical_history': 'Select * from Medication', 'search_pharmacy_by_name': 'Select Name, POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) AS Distance from Hospitals order by distance limit 5', 'search_hospital_by_name': 'Select Name from Hospitals order by POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) limit 5'}



127.0.0.1 - - [06/Nov/2020 07:16:05] "GET /get?msg=i%20need%20some%20help%20to%20use%20this%20application HTTP/1.1" 200 -


[{'intent': 'assistance_required', 'probability': '1.0'}]
[{'intent': 'assistance_required', 'probability': '1.0'}]
tag:['dispatch_team']
[{'intent': 'assistance_required', 'probability': '1.0'}]
{'medication_module': 'Select * from Medication where Duration_in_weeks>3', 'search_medical_history': 'Select * from Medication', 'search_pharmacy_by_name': 'Select Name, POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) AS Distance from Hospitals order by distance limit 5', 'search_hospital_by_name': 'Select Name from Hospitals order by POWER((POWER((Latitude-36.3361588500001),2)+POWER((Longitude-119.645667298),2)),0.5) limit 5'}

